In [1]:
#Based on https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/2_BasicModels/linear_regression.ipynb
import tensorflow as tf
import numpy
import csv
rng = numpy.random

In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 300
display_step = 50

In [3]:
#Training data
csv = numpy.genfromtxt("solves.csv", skip_header=1, delimiter=",")
train_difficulty = csv[:, 0]
train_features = csv[:, 1:]

In [4]:
#number of training samples
n_samples = train_difficulty.shape[0]

#number of features
feature_length = train_features.shape[1]

In [5]:
#Normalize all of the columns to be between -0.5 and 0.5 with a smooth distribution
#TODO: we need to save the maxVal so other folks in the model can normalize appropriately

#TODO: figure out a better way to normalize these; right now many of them are squishing at -0.5 since their
#distributions are often very left-skewed
for colNum in xrange(0, feature_length):
    col = train_features[:,colNum]
    maxVal = max(col)
    if maxVal == 0:
        maxVal = 1
    for index in xrange(0, len(col)):
        col[index] = (col[index] - (maxVal / 2)) / maxVal

In [6]:

# tf Graph Input
Difficulty = tf.placeholder("float")
Features = tf.placeholder("float", shape=(feature_length))

# Set model weights
W = tf.Variable(tf.random_normal([feature_length], stddev=0.25), name="weight")
b = tf.Variable(0.0, name="bias")

In [7]:
# Construct a linear model
pred = tf.add(tf.mul(Features, W), b)

In [8]:
#TODO: calculate R2 and output it over time
# Mean squared error
with tf.name_scope("cost") as scope:
    cost = tf.reduce_sum(tf.pow(pred-Difficulty, 2))/(2*n_samples)
    cost_summ = tf.scalar_summary("cost", cost)
# Gradient descent
with tf.name_scope("optimize") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [9]:
#Summary ops to collect data
#The use of tensorboard is adapted from the example on https://www.tensorflow.org/versions/r0.7/how_tos/summaries_and_tensorboard/index.html
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
difficulty_hist = tf.histogram_summary("difficulty", Difficulty)

In [10]:
# Initializing the variables
init = tf.initialize_all_variables()

In [11]:
# Launch the graph
with tf.Session() as sess:
    
    merged = tf.merge_all_summaries()
    #TODO: figure out how to get these to overwrite existing logs (or at least be separate)
    writer = tf.train.SummaryWriter("tmp/linear_logs", sess.graph_def)
    
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (features, difficulty) in zip(train_features, train_difficulty):
            sess.run(optimizer, feed_dict={Features: features, Difficulty: difficulty})

        #Display logs per epoch step
        if (epoch+1) % display_step == 0:
            result = sess.run([merged,cost], feed_dict={Features: features, Difficulty: difficulty})
            summary_str = result[0]
            c = result[1]
            writer.add_summary(summary_str, epoch)
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b)

    print "Optimization Finished!"
    training_cost = sess.run(cost, feed_dict={Features: features, Difficulty: difficulty})
    print "Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n'


Epoch: 0050 cost= 0.007192562 W= [-0.04834342 -0.13540412  0.35155329 -0.42981386  0.23225115  0.49575615
  0.24197435  0.15733233  0.17822041  0.05280761  0.33208877  0.34243354
  0.28202844  0.05939834 -0.30064267 -0.01703261 -0.18630184 -0.23021659
 -0.08854297  0.04416275  0.12727964 -0.25188053 -0.06973174  0.60302627
  0.00253241 -0.13842016  0.3072648  -0.23878251  0.06400895 -0.57291526
 -0.16307452  0.03871728  0.20529599  0.08184347 -0.1058052  -0.43325245
  0.43862256 -0.30968469  0.22350393 -0.23691283  0.04596709  0.35992762
 -0.05019029 -0.11165523 -0.05233024  0.05789621  0.06276163  0.04046629
 -0.36277628 -0.0810663  -0.14070025 -0.18129534 -0.01439438  0.00069876
  0.18140323 -0.33203843 -0.152356   -0.17979912  0.15622851 -0.1982532
 -0.05431211  0.5803113   0.07454147 -0.25211942 -0.15584818  0.01258918
  0.02979302 -0.06814001 -0.22866897 -0.08105121 -0.19036575  0.08265972
  0.26668501 -0.12163173 -0.27324539  0.20305881  0.01028989 -0.52156901
  0.03103366  0.002

In [12]:
#TODO: output the model with bias, cost, and weights all zipped up with their names